### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import collections

# File to Load (Remember to Change These)
file_to_load = "./Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)
pd.options.display.float_format = '${:,.2f}'.format

df.head()

## Player Count

* Display the total number of players


In [ ]:
# Calculate the number of unique players in the DataFrame
lst_players = len(df["SN"].unique())
lst_players

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [ ]:
#Format with panda options : pd.options.display.float_format = '${:,.2f}'.format

tot_item = len(df["Item ID"].unique())
avg_price = round(df["Price"].mean(),2)
purch_num = len(df["Purchase ID"])
total_rev = df["Price"].sum()

summary_table = pd.DataFrame({"Number of unique items": [tot_item],
                              "Average Price":'${:,.2f}'.format(avg_price),
                              "Number of Purchases" :[purch_num],
                              "Total Revenue":'${:,.2f}'.format(total_rev)})
summary_table

## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [ ]:
#drop_duplicates for users with multiple purchases. We want to have unique users
df_unique = df.drop_duplicates("SN")
df_unique

#Creating individual columns for Male/Female/Other categories

male_perc = len(df_unique.loc[(df_unique["Gender"]== "Male")])/len(df_unique["Purchase ID"])
male_count = (df_unique["Gender"]== "Male").sum()
#print('{:,.2f}%'.format(100*male_perc), male_count)

fem_perc = len(df_unique.loc[(df_unique["Gender"]== "Female")])/len(df_unique["Purchase ID"])
fem_count = (df_unique["Gender"]== "Female").sum()
#print('{:,.2f}%'.format(100*fem_perc), fem_count)

oth_perc = len(df_unique.loc[(df_unique["Gender"]!= "Male") & (df_unique["Gender"]!= "Female")])/len(df_unique["Purchase ID"])
oth_count = ((df_unique["Gender"]!= "Male") & (df_unique["Gender"]!= "Female")).sum()
#print('{:,.2f}%'.format(100*oth_perc), oth_count)


data = {'Gender':['Female','Male','Other / Non-Disclosed'],
        'Total Count':[fem_count,male_count,oth_count],
       'Percentage of Players':['{:,.2f}%'.format(fem_perc*100), '{:,.2f}%'.format(male_perc*100), '{:,.2f}%'.format(oth_perc*100)]
       }
df_test =pd.DataFrame(data)

df_test


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [ ]:
#Groupby Gender
df_new = df.groupby(["Gender"], sort=True, as_index=False).count()
df_new = df_new.drop(columns = ['SN','Age','Item ID','Item Name','Price'])

df_new.rename(columns={'Purchase ID':'Purchase Count'}) 
#Conitnue building dataframe df_new


df1 = df.groupby(['Gender']).agg({'Purchase ID':'count',
                                  'Price':['mean','sum',],
                                 })

#Join multiindex column names with '_'
df1.columns = df1.columns.map('_'.join)

#Merge columns
dfm = pd.merge(df1, df_test, left_on = 'Gender', right_on = 'Gender')

#Calculate new column/ Results are stored in a new column
dfm['Result'] = dfm['Price_sum']/dfm['Total Count']
#Drop redundant columns
dfm = dfm.drop(['Total Count', 'Percentage of Players'], axis=1)

dfm =dfm.rename(columns= {'Purchase ID_count':'Purchase Count', 
                     'Price_mean':'Average Purchase Price', 
                     'Price_sum':'Total Purchase Value', 
                     'Result': 'Avg Total Purchase per Person'} )
dfm



## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [ ]:
#Create bins
bins = [0,10,15,20,25,30,35,40,150]
group_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

#Groupby unique purchases table. We need that for the last column 'Avg Purchase per Person' and 'Percentage of Players'
df_unique["Age Group"] = pd.cut(df_unique["Age"], bins,right=False, labels=group_names)
df_unique = df_unique.groupby(['Age Group']).agg({'Purchase ID':'count',
                                                  'Price':['mean','sum']
                                                  })
#Prepearing Age Demographics dataframe/ Converting float2string and minimize decimal. Use round() 
df_unique['Percentage of Players']= df_unique['Purchase ID']/len(df["SN"].unique())
df_unique['Percentage of Players'] = (df_unique['Percentage of Players'] * 100).round(2).astype(str) + '%'

df_unique

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [ ]:
#Purchase Analysis by Age. Some parts are used from the previous 'Age Demographics' part

df_age = df

#Groupby Age
df_age["Age Group"] = pd.cut(df["Age"], bins,right=False, labels=group_names)
df_age = df_age.groupby(['Age Group']).agg({'Purchase ID':'count',
                'Price':['mean','sum']
                  })

df_unique.columns
df_age = pd.concat([df_age, df_unique[['Purchase ID']]], axis=1)


#Column calculation and joining
df_age.columns = df_age.columns.map('_'.join)
df_age.columns = ['Purchase Count', 'Average Purchase Price','Total Purchase Value', 'Unique Purchase per Person']
df_age.columns
df_age['Avg Total Purchase per Person'] = df_age['Total Purchase Value']/df_age['Unique Purchase per Person']
del df_age['Unique Purchase per Person']
df_age


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [ ]:
#Top Spenders
df_sn = df
df_sn = df_sn.groupby(['SN']).agg({'Purchase ID':'count',
                'Price':['mean','sum']
                  })
df_sn.columns = df_sn.columns.map('_'.join)
df_sn.nlargest(5,'Price_sum')


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [ ]:
#Most Popular items
df_pop = df
df_pop = df_pop.groupby(['Item ID', 'Item Name']).agg({'Purchase ID':'count',
                'Price':['mean','sum']
                  })
df_pop.columns = df_pop.columns.map('_'.join)

df_pop.nlargest(5,'Purchase ID_count')

## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [ ]:
#Most Profitable Items
df_pop.nlargest(5,'Price_sum')